In [1]:
# x-enVent
import pandas as pd
import os

df = pd.read_json('../data/x-enVENT-TroianoOberlaenderWeggeKlinger-LREC2022-corpus.json')
appraisals = df['annotations'][0]['aggregated'][0]['experiencers'][0]['appraisals'].keys()
df

,id,text,tokens,annotations,original_dataset,original_emotion
0,7520,I felt ... when I passed my exams. I had worke...,"[I, felt, ., ., ., when, I, passed, my, exams,...","{'aggregated': [{'id': '7520#', 'experiencers'...",enISEAR,joy
1,8288,I felt ... when I lost a cherished piece of je...,"[I, felt, ., ., ., when, I, lost, a, cherished...","{'aggregated': [{'id': '8288#', 'experiencers'...",enISEAR,sadness
2,7820,I felt ... because there was a very loud and u...,"[I, felt, ., ., ., because, there, was, a, ver...","{'aggregated': [{'id': '7820#', 'experiencers'...",enISEAR,fear
3,7908,I felt...because someone was yelling at me in ...,"[I, felt, ., ., ., because, someone, was, yell...","{'aggregated': [{'id': '7908#', 'experiencers'...",enISEAR,fear
4,7683,I felt ... when I received a postage stamp tha...,"[I, felt, ., ., ., when, I, received, a, posta...","{'aggregated': [{'id': '7683#', 'experiencers'...",enISEAR,joy
...,...,...,...,...,...,...
715,7810,I felt ... when I thought I had lost my wallet.,"[I, felt, ., ., ., when, I, thought, I, had, l...","{'aggregated': [{'id': '7810#', 'experiencers'...",enISEAR,fear
716,7796,I felt ... when I was climbing up a very high ...,"[I, felt, ., ., ., when, I, was, climbing, up,...","{'aggregated': [{'id': '7796#', 'experiencers'...",enISEAR,fear
717,7801,I felt ... that another teenage child had been...,"[I, felt, ., ., ., that, another, teenage, chi...","{'aggregated': [{'id': '7801#', 'experiencers'...",enISEAR,anger
718,3707,When my sister's boyfriend accused her of havi...,"[When, my, sister's, boyfriend, accused, her, ...","{'aggregated': [{'id': '3707#', 'experiencers'...",ISEAR,disgust


In [2]:
appraisals

dict_keys(['suddenness', 'familiarity', 'pleasantness', 'understand', 'goal_relevance', 'self_responsibility', 'other_responsibility', 'situational_responsibility', 'effort', 'exert', 'attend', 'consider', 'outcome_probability', 'expectation_discrepancy', 'goal_conduciveness', 'urgency', 'self_control', 'other_control', 'situational_control', 'adjustment_check', 'internal_check', 'external_check'])

In [3]:
def xenvent_reader(fpath):
    df = pd.read_json(fpath)
    appraisal_list = df['annotations'][0]['aggregated'][0]['experiencers'][0]['appraisals'].keys()
    scores = {}
    events = []
    for ix in range(len(df)):
        for app in appraisal_list:
            score = (df['annotations'][ix]['aggregated'][0]['experiencers'][0]['appraisals'][app]['avg'])
            if app in scores:
                scores[app].append(score)
            else:
                scores[app] = [score]
        events.append(df['annotations'][ix]['aggregated'][0]['events'][0]['text'])
    df['text'] = df.text.str.replace(r'\w+(\.{3,})', 'felt ... ', regex=True)
    df['text']= df.apply(lambda x: x['text'].replace('felt ...', 'felt ' + str(x['original_emotion'])), axis=1)
    appraisals = pd.DataFrame(scores)
    for app in appraisals:
        df[app] = list(appraisals[app])
    cols = ['id', 'text'] + list(appraisals.columns.values) + ['original_emotion']
    df = df[cols]
    return df

In [4]:
df1 = xenvent_reader('../data/x-enVENT-TroianoOberlaenderWeggeKlinger-LREC2022-corpus.json')
df1

,id,text,suddenness,familiarity,pleasantness,understand,goal_relevance,self_responsibility,other_responsibility,situational_responsibility,...,expectation_discrepancy,goal_conduciveness,urgency,self_control,other_control,situational_control,adjustment_check,internal_check,external_check,original_emotion
0,7520,I felt joy when I passed my exams. I had worke...,3.750000,2.500000,4.750000,4.750000,2.000000,5.000000,1.25,1.750000,...,2.500000,4.50,1.250000,3.750000,1.000000,2.750000,5.000000,1.000000,1.000000,joy
1,8288,I felt sadness when I lost a cherished piece o...,4.500000,1.500000,1.750000,4.750000,2.250000,4.500000,1.00,2.500000,...,4.750000,1.00,2.000000,2.500000,1.000000,4.500000,3.250000,2.750000,1.250000,sadness
2,7820,I felt fear because there was a very loud and ...,5.000000,2.000000,1.500000,1.500000,2.500000,1.000000,2.25,4.250000,...,5.000000,1.00,2.500000,1.500000,2.000000,4.250000,3.250000,2.250000,1.250000,fear
3,7908,I felt fear because someone was yelling at me ...,4.750000,1.750000,1.750000,4.000000,2.750000,2.250000,5.00,1.000000,...,4.500000,1.25,2.000000,1.750000,4.500000,1.000000,3.250000,3.250000,2.500000,fear
4,7683,I felt joy when I received a postage stamp tha...,3.250000,2.000000,4.500000,4.750000,2.000000,3.500000,3.00,1.750000,...,1.500000,4.25,1.500000,3.250000,1.250000,2.250000,4.750000,1.000000,1.000000,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,7810,I felt fear when I thought I had lost my wallet.,4.500000,1.750000,2.000000,3.000000,2.000000,4.000000,1.00,2.250000,...,4.750000,1.25,2.500000,2.250000,1.000000,4.500000,3.000000,3.000000,1.250000,fear
716,7796,I felt fear when I was climbing up a very high...,2.666667,1.666667,1.666667,4.000000,4.000000,4.333333,1.00,2.333333,...,3.666667,1.00,1.666667,3.666667,1.666667,3.666667,2.666667,2.666667,1.000000,fear
717,7801,I felt anger that another teenage child had be...,4.000000,1.750000,1.250000,4.000000,2.500000,1.000000,5.00,1.000000,...,4.750000,1.00,3.000000,1.250000,4.500000,1.750000,3.000000,4.000000,3.750000,anger
718,3707,When my sister's boyfriend accused her of havi...,4.250000,1.000000,1.250000,3.750000,1.750000,1.000000,5.00,1.000000,...,4.500000,1.00,1.250000,1.000000,5.000000,1.000000,3.750000,3.500000,1.750000,disgust


In [5]:
df1 = df1.astype({col: int for col in df1.columns[2:-1]})
df1

,id,text,suddenness,familiarity,pleasantness,understand,goal_relevance,self_responsibility,other_responsibility,situational_responsibility,...,expectation_discrepancy,goal_conduciveness,urgency,self_control,other_control,situational_control,adjustment_check,internal_check,external_check,original_emotion
0,7520,I felt joy when I passed my exams. I had worke...,3,2,4,4,2,5,1,1,...,2,4,1,3,1,2,5,1,1,joy
1,8288,I felt sadness when I lost a cherished piece o...,4,1,1,4,2,4,1,2,...,4,1,2,2,1,4,3,2,1,sadness
2,7820,I felt fear because there was a very loud and ...,5,2,1,1,2,1,2,4,...,5,1,2,1,2,4,3,2,1,fear
3,7908,I felt fear because someone was yelling at me ...,4,1,1,4,2,2,5,1,...,4,1,2,1,4,1,3,3,2,fear
4,7683,I felt joy when I received a postage stamp tha...,3,2,4,4,2,3,3,1,...,1,4,1,3,1,2,4,1,1,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,7810,I felt fear when I thought I had lost my wallet.,4,1,2,3,2,4,1,2,...,4,1,2,2,1,4,3,3,1,fear
716,7796,I felt fear when I was climbing up a very high...,2,1,1,4,4,4,1,2,...,3,1,1,3,1,3,2,2,1,fear
717,7801,I felt anger that another teenage child had be...,4,1,1,4,2,1,5,1,...,4,1,3,1,4,1,3,4,3,anger
718,3707,When my sister's boyfriend accused her of havi...,4,1,1,3,1,1,5,1,...,4,1,1,1,5,1,3,3,1,disgust


In [6]:
df2 = xenvent_reader('../data/x-enVENT-additional-180.json')
df2

,id,text,suddenness,familiarity,pleasantness,understand,goal_relevance,self_responsibility,other_responsibility,situational_responsibility,...,expectation_discrepancy,goal_conduciveness,urgency,self_control,other_control,situational_control,adjustment_check,internal_check,external_check,original_emotion
0,7760,I felt anger when someone made a racist commen...,5.0,3.0,2.0,5.0,1.0,1.0,5.0,1.0,...,5.0,1.0,2.0,3.0,3.0,1.0,4.0,3.0,5.0,anger
1,8038,I felt anger when the neighbours’ son smashed ...,4.0,2.0,2.0,5.0,2.0,1.0,5.0,1.0,...,5.0,1.0,2.0,2.0,4.0,1.0,4.0,5.0,4.0,anger
2,7531,I felt guilt when I couldn't visit a relative ...,2.0,3.0,2.0,5.0,3.0,1.0,1.0,5.0,...,5.0,1.0,1.0,2.0,1.0,4.0,4.0,3.0,1.0,guilt
3,8036,I felt sadness when I saw how migrants die cro...,2.0,3.0,1.0,5.0,1.0,1.0,5.0,5.0,...,5.0,1.0,2.0,2.0,5.0,5.0,3.0,5.0,5.0,sadness
4,8153,I felt shame that I lied about my employment e...,1.0,3.0,2.0,5.0,2.0,5.0,1.0,1.0,...,1.0,5.0,1.0,3.0,3.0,3.0,4.0,2.0,3.0,shame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5.0,1.0,1.0,5.0,4.0,1.0,5.0,1.0,...,5.0,1.0,4.0,2.0,4.0,1.0,2.0,5.0,5.0,fear
176,7804,I felt disgust when I opened a pea shell and f...,5.0,2.0,2.0,5.0,1.0,2.0,1.0,5.0,...,5.0,1.0,2.0,5.0,1.0,1.0,4.0,3.0,1.0,disgust
177,7675,I felt joy dancing down Gloucester Place in Lo...,2.0,2.0,5.0,5.0,3.0,5.0,5.0,1.0,...,2.0,5.0,1.0,3.0,3.0,1.0,5.0,1.0,1.0,joy
178,8162,I felt sadness when my Uncle died last August....,2.0,2.0,1.0,5.0,3.0,1.0,1.0,5.0,...,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,1.0,sadness


In [7]:
df2 = df2.astype({col: int for col in df2.columns[2:-1]})
df2

,id,text,suddenness,familiarity,pleasantness,understand,goal_relevance,self_responsibility,other_responsibility,situational_responsibility,...,expectation_discrepancy,goal_conduciveness,urgency,self_control,other_control,situational_control,adjustment_check,internal_check,external_check,original_emotion
0,7760,I felt anger when someone made a racist commen...,5,3,2,5,1,1,5,1,...,5,1,2,3,3,1,4,3,5,anger
1,8038,I felt anger when the neighbours’ son smashed ...,4,2,2,5,2,1,5,1,...,5,1,2,2,4,1,4,5,4,anger
2,7531,I felt guilt when I couldn't visit a relative ...,2,3,2,5,3,1,1,5,...,5,1,1,2,1,4,4,3,1,guilt
3,8036,I felt sadness when I saw how migrants die cro...,2,3,1,5,1,1,5,5,...,5,1,2,2,5,5,3,5,5,sadness
4,8153,I felt shame that I lied about my employment e...,1,3,2,5,2,5,1,1,...,1,5,1,3,3,3,4,2,3,shame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5,1,1,5,4,1,5,1,...,5,1,4,2,4,1,2,5,5,fear
176,7804,I felt disgust when I opened a pea shell and f...,5,2,2,5,1,2,1,5,...,5,1,2,5,1,1,4,3,1,disgust
177,7675,I felt joy dancing down Gloucester Place in Lo...,2,2,5,5,3,5,5,1,...,2,5,1,3,3,1,5,1,1,joy
178,8162,I felt sadness when my Uncle died last August....,2,2,1,5,3,1,1,5,...,1,1,1,1,1,5,3,5,1,sadness


In [8]:
df_xenvent = pd.concat([df1, df2])
df_xenvent

,id,text,suddenness,familiarity,pleasantness,understand,goal_relevance,self_responsibility,other_responsibility,situational_responsibility,...,expectation_discrepancy,goal_conduciveness,urgency,self_control,other_control,situational_control,adjustment_check,internal_check,external_check,original_emotion
0,7520,I felt joy when I passed my exams. I had worke...,3,2,4,4,2,5,1,1,...,2,4,1,3,1,2,5,1,1,joy
1,8288,I felt sadness when I lost a cherished piece o...,4,1,1,4,2,4,1,2,...,4,1,2,2,1,4,3,2,1,sadness
2,7820,I felt fear because there was a very loud and ...,5,2,1,1,2,1,2,4,...,5,1,2,1,2,4,3,2,1,fear
3,7908,I felt fear because someone was yelling at me ...,4,1,1,4,2,2,5,1,...,4,1,2,1,4,1,3,3,2,fear
4,7683,I felt joy when I received a postage stamp tha...,3,2,4,4,2,3,3,1,...,1,4,1,3,1,2,4,1,1,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5,1,1,5,4,1,5,1,...,5,1,4,2,4,1,2,5,5,fear
176,7804,I felt disgust when I opened a pea shell and f...,5,2,2,5,1,2,1,5,...,5,1,2,5,1,1,4,3,1,disgust
177,7675,I felt joy dancing down Gloucester Place in Lo...,2,2,5,5,3,5,5,1,...,2,5,1,3,3,1,5,1,1,joy
178,8162,I felt sadness when my Uncle died last August....,2,2,1,5,3,1,1,5,...,1,1,1,1,1,5,3,5,1,sadness


In [9]:
# crowd-envent
df_crowdenvent = pd.read_csv('../data/crowd-enVent-train.tsv', sep='\t')
df_crowdenvent

,Unnamed: 0,round_number,emotion,text_id,timestamp,prolific_id,anger,boredom,disgust,fear,...,dependable,anxious,open,quiet,sympathetic,disorganized,calm,conventional,did_you_lie?,original_demographics
0,2550,round-3,joy,3573,2021/09/03 9:28:01 pm EET,8f7e7167f32371004cbf5e3e92bbcb89,1,1,1,1,...,5.0,5.0,5.0,4.0,3.0,4.0,3.0,3.0,--,ORIGINAL
1,2746,round-5,joy,51199,2021/10/06 2:23:33 pm EET,034bb12696eb7f4e7b155459c1d5fd57,1,2,1,1,...,3.0,7.0,5.0,2.0,5.0,6.0,1.0,4.0,The event really happened in my life.,COPIED FROM: text id 5296
2,3539,round-6,no-emotion,6601,2021/11/01 10:02:18 pm CET,0743877ee40f4b9ec5bf92195381ed2b,1,1,1,1,...,5.0,1.0,7.0,6.0,6.0,2.0,6.0,1.0,The event really happened in my life.,ORIGINAL
3,3127,round-3,no-emotion,3727,2021/09/03 10:12:28 pm EET,4d82215ac84dc6962609bce7dde2ab5e,1,2,2,1,...,6.0,6.0,4.0,7.0,7.0,2.0,5.0,3.0,--,ORIGINAL
4,5390,round-5,shame,52291,2021/10/06 1:36:21 pm EET,5ef572df9b4b7b5e890c3f594abcf111,1,2,1,1,...,2.0,4.0,4.0,5.0,4.0,6.0,3.0,2.0,The event really happened in my life.,COPIED FROM: text id 593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2195,round-7,fear,760,2021/11/09 10:54:00 pm CET,3bde2e5473c55aac3513e7cdde442058,1,1,1,1,...,3.0,4.0,6.0,3.0,6.0,4.0,3.0,2.0,The event really happened in my life.,COPIED FROM: text id 73
4316,435,round-5,anger,5121,2021/10/06 2:58:10 pm EET,75ff2441794c680313a00fc96c8aa1fa,1,2,1,1,...,5.0,6.0,7.0,5.0,6.0,3.0,5.0,5.0,The event really happened in my life.,COPIED FROM: text id 5535
4317,1809,round-4,fear,4506,2021/09/20 1:24:06 am EET,dbdcf55b72c3eeb6828f6717be44d89f,1,5,1,1,...,5.0,6.0,6.0,5.0,5.0,6.0,3.0,5.0,--,COPIED FROM: text id 41892
4318,5776,round-5,surprise,52508,2021/10/06 2:34:44 pm EET,add1c4bd0a37a9e3d44872e44985131d,1,1,1,1,...,5.0,6.0,2.5,4.0,4.5,4.5,3.0,4.0,The event really happened in my life.,"AVERAGED: text id 5832, 51570"


In [10]:
df_crowdenvent.keys()

Index(['Unnamed: 0', 'round_number', 'emotion', 'text_id', 'timestamp',
       'prolific_id', 'anger', 'boredom', 'disgust', 'fear', 'guilt', 'joy',
       'pride', 'relief', 'sadness', 'shame', 'surprise', 'trust',
       'generated_text', 'hidden_emo_text', 'event_duration',
       'emotion_duration', 'intensity', 'confidence', 'suddenness',
       'familiarity', 'predict_event', 'pleasantness', 'unpleasantness',
       'goal_relevance', 'chance_responsblt', 'self_responsblt',
       'other_responsblt', 'predict_conseq', 'goal_support', 'urgency',
       'self_control', 'other_control', 'chance_control', 'accept_conseq',
       'standards', 'social_norms', 'attention', 'not_consider', 'effort',
       'previous_participation', 'age', 'gender', 'education', 'ethnicity',
       'extravert', 'critical', 'dependable', 'anxious', 'open', 'quiet',
       'sympathetic', 'disorganized', 'calm', 'conventional', 'did_you_lie?',
       'original_demographics'],
      dtype='object')

In [11]:
df_crowdenvent = df_crowdenvent[['text_id', 'generated_text', 'suddenness',
       'familiarity', 'predict_event', 'pleasantness', 'unpleasantness',
       'goal_relevance', 'chance_responsblt', 'self_responsblt',
       'other_responsblt', 'predict_conseq', 'goal_support', 'urgency',
       'self_control', 'other_control', 'chance_control', 'accept_conseq',
       'standards', 'social_norms', 'attention', 'not_consider', 'effort', 'emotion']]

In [12]:
df_xenvent.keys()

Index(['id', 'text', 'suddenness', 'familiarity', 'pleasantness', 'understand',
       'goal_relevance', 'self_responsibility', 'other_responsibility',
       'situational_responsibility', 'effort', 'exert', 'attend', 'consider',
       'outcome_probability', 'expectation_discrepancy', 'goal_conduciveness',
       'urgency', 'self_control', 'other_control', 'situational_control',
       'adjustment_check', 'internal_check', 'external_check',
       'original_emotion'],
      dtype='object')

In [13]:
keys = { 'id' : 'text_id',
        'text' : 'generated_text',
        'suddenness' : 'suddenness',
        'familiarity' : 'familiarity',
        'pleasantness' : 'pleasantness',
        'goal_relevance' : 'goal_relevance',
        'goal_conduciveness' : 'goal_support',
        'self_responsibility' : 'self_responsblt',
        'other_responsibility' : 'other_responsblt',
        'situational_responsibility' : 'chance_responsblt',
        'expectation_discrepancy' : 'predict_conseq',
        'outcome_probability' : 'predict_event',
        'attend' : 'attention',
        'urgency': 'urgency',
        'self_control': 'self_control',
        'other_control': 'other_control',
        'situational_control' : 'chance_control',
        'internal_check' : 'standards',
        'external_check' : 'social_norms',
        'effort': 'effort'
}
keys

{'id': 'text_id',
 'text': 'generated_text',
 'suddenness': 'suddenness',
 'familiarity': 'familiarity',
 'pleasantness': 'pleasantness',
 'goal_relevance': 'goal_relevance',
 'goal_conduciveness': 'goal_support',
 'self_responsibility': 'self_responsblt',
 'other_responsibility': 'other_responsblt',
 'situational_responsibility': 'chance_responsblt',
 'expectation_discrepancy': 'predict_conseq',
 'outcome_probability': 'predict_event',
 'attend': 'attention',
 'urgency': 'urgency',
 'self_control': 'self_control',
 'other_control': 'other_control',
 'situational_control': 'chance_control',
 'internal_check': 'standards',
 'external_check': 'social_norms',
 'effort': 'effort'}

In [14]:
inv_keys = {v : k for k, v in keys.items()}
inv_keys

{'text_id': 'id',
 'generated_text': 'text',
 'suddenness': 'suddenness',
 'familiarity': 'familiarity',
 'pleasantness': 'pleasantness',
 'goal_relevance': 'goal_relevance',
 'goal_support': 'goal_conduciveness',
 'self_responsblt': 'self_responsibility',
 'other_responsblt': 'other_responsibility',
 'chance_responsblt': 'situational_responsibility',
 'predict_conseq': 'expectation_discrepancy',
 'predict_event': 'outcome_probability',
 'attention': 'attend',
 'urgency': 'urgency',
 'self_control': 'self_control',
 'other_control': 'other_control',
 'chance_control': 'situational_control',
 'standards': 'internal_check',
 'social_norms': 'external_check',
 'effort': 'effort'}

In [15]:
df_crowdenvent = df_crowdenvent[list(keys.values())]
df_crowdenvent

,text_id,generated_text,suddenness,familiarity,pleasantness,goal_relevance,goal_support,self_responsblt,other_responsblt,chance_responsblt,predict_conseq,predict_event,attention,urgency,self_control,other_control,chance_control,standards,social_norms,effort
0,3573,I was told “I love you” by a special person,2,2,5,5,5,3,3,3,4,4,4,5,5,5,5,1,1,3
1,51199,I gave birth to my son.,2,3,3,5,4,5,5,4,5,5,5,5,1,2,4,1,1,5
2,6601,When a student told me to ‘fuck off’ to my face,5,1,1,1,1,1,5,1,1,1,5,5,1,5,1,3,5,5
3,3727,When I’m doing my day to day routine.,4,2,1,2,2,5,4,1,2,2,1,1,4,1,4,1,1,4
4,52291,i failed a subject.,2,3,1,5,1,4,3,3,4,4,4,2,4,4,4,4,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,760,I had to go into my brothers flooded basement ...,3,1,1,1,1,1,2,4,1,1,5,4,4,1,5,4,1,3
4316,5121,When my dad cut me from his life,2,1,1,5,1,1,5,1,1,1,5,5,1,5,1,4,5,5
4317,4506,I felt fear when walking home alone on Wednesd...,2,4,1,1,1,5,1,1,5,4,5,1,3,3,1,5,1,4
4318,52508,my husband and my daughter came to meet me fro...,3,4,4,2,2,1,5,1,1,2,1,3,1,5,1,1,1,1


In [16]:
df_xenvent = df_xenvent[list(keys.keys())]
df_xenvent

,id,text,suddenness,familiarity,pleasantness,goal_relevance,goal_conduciveness,self_responsibility,other_responsibility,situational_responsibility,expectation_discrepancy,outcome_probability,attend,urgency,self_control,other_control,situational_control,internal_check,external_check,effort
0,7520,I felt joy when I passed my exams. I had worke...,3,2,4,2,4,5,1,1,2,4,1,1,3,1,2,1,1,1
1,8288,I felt sadness when I lost a cherished piece o...,4,1,1,2,1,4,1,2,4,3,1,2,2,1,4,2,1,2
2,7820,I felt fear because there was a very loud and ...,5,2,1,2,1,1,2,4,5,1,4,2,1,2,4,2,1,3
3,7908,I felt fear because someone was yelling at me ...,4,1,1,2,1,2,5,1,4,2,2,2,1,4,1,3,2,3
4,7683,I felt joy when I received a postage stamp tha...,3,2,4,2,4,3,3,1,1,4,1,1,3,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5,1,1,4,1,1,5,1,5,1,4,4,2,4,1,5,5,3
176,7804,I felt disgust when I opened a pea shell and f...,5,2,2,1,1,2,1,5,5,5,3,2,5,1,1,3,1,1
177,7675,I felt joy dancing down Gloucester Place in Lo...,2,2,5,3,5,5,5,1,2,5,1,1,3,3,1,1,1,1
178,8162,I felt sadness when my Uncle died last August....,2,2,1,3,1,1,1,5,1,4,1,1,1,1,5,5,1,3


In [17]:
df_xenvent.rename(columns=keys, inplace=True)

/tmp/ipykernel_631030/4027659132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xenvent.rename(columns=keys, inplace=True)


In [18]:
df_xenvent

,text_id,generated_text,suddenness,familiarity,pleasantness,goal_relevance,goal_support,self_responsblt,other_responsblt,chance_responsblt,predict_conseq,predict_event,attention,urgency,self_control,other_control,chance_control,standards,social_norms,effort
0,7520,I felt joy when I passed my exams. I had worke...,3,2,4,2,4,5,1,1,2,4,1,1,3,1,2,1,1,1
1,8288,I felt sadness when I lost a cherished piece o...,4,1,1,2,1,4,1,2,4,3,1,2,2,1,4,2,1,2
2,7820,I felt fear because there was a very loud and ...,5,2,1,2,1,1,2,4,5,1,4,2,1,2,4,2,1,3
3,7908,I felt fear because someone was yelling at me ...,4,1,1,2,1,2,5,1,4,2,2,2,1,4,1,3,2,3
4,7683,I felt joy when I received a postage stamp tha...,3,2,4,2,4,3,3,1,1,4,1,1,3,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5,1,1,4,1,1,5,1,5,1,4,4,2,4,1,5,5,3
176,7804,I felt disgust when I opened a pea shell and f...,5,2,2,1,1,2,1,5,5,5,3,2,5,1,1,3,1,1
177,7675,I felt joy dancing down Gloucester Place in Lo...,2,2,5,3,5,5,5,1,2,5,1,1,3,3,1,1,1,1
178,8162,I felt sadness when my Uncle died last August....,2,2,1,3,1,1,1,5,1,4,1,1,1,1,5,5,1,3


In [19]:
df_train = pd.concat([df_crowdenvent, df_xenvent])
df_train

,text_id,generated_text,suddenness,familiarity,pleasantness,goal_relevance,goal_support,self_responsblt,other_responsblt,chance_responsblt,predict_conseq,predict_event,attention,urgency,self_control,other_control,chance_control,standards,social_norms,effort
0,3573,I was told “I love you” by a special person,2,2,5,5,5,3,3,3,4,4,4,5,5,5,5,1,1,3
1,51199,I gave birth to my son.,2,3,3,5,4,5,5,4,5,5,5,5,1,2,4,1,1,5
2,6601,When a student told me to ‘fuck off’ to my face,5,1,1,1,1,1,5,1,1,1,5,5,1,5,1,3,5,5
3,3727,When I’m doing my day to day routine.,4,2,1,2,2,5,4,1,2,2,1,1,4,1,4,1,1,4
4,52291,i failed a subject.,2,3,1,5,1,4,3,3,4,4,4,2,4,4,4,4,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,7861,I felt fear when I saw someone else got stabbe...,5,1,1,4,1,1,5,1,5,1,4,4,2,4,1,5,5,3
176,7804,I felt disgust when I opened a pea shell and f...,5,2,2,1,1,2,1,5,5,5,3,2,5,1,1,3,1,1
177,7675,I felt joy dancing down Gloucester Place in Lo...,2,2,5,3,5,5,5,1,2,5,1,1,3,3,1,1,1,1
178,8162,I felt sadness when my Uncle died last August....,2,2,1,3,1,1,1,5,1,4,1,1,1,1,5,5,1,3


In [20]:
df_train.to_csv('./data/train.csv')

In [21]:
df_test = pd.read_csv('../data/crowd-enVent-test.tsv', sep = '\t')
df_test = df_test[list(keys.values())]
df_test.to_csv('./data/test.csv')

In [22]:
df_val = pd.read_csv('../data/crowd-enVent-val.tsv', sep = '\t')
df_val = df_val[list(keys.values())]
df_val.to_csv('./data/val.csv')